In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("Atipico1/mrqa_v2_unans")["train"]

In [3]:
squad = load_dataset("rajpurkar/squad_v2")

In [4]:
from datasets import concatenate_datasets
total_squad = concatenate_datasets([squad["train"], squad["validation"]])

In [47]:
total_squad = total_squad.filter(lambda x : x["answers"]["text"] == [])
total_squad = total_squad.filter(lambda x: len(x["context"].split()) < 120)

Filter:   0%|          | 0/49443 [00:00<?, ? examples/s]

Filter:   0%|          | 0/49443 [00:00<?, ? examples/s]

In [10]:
# from sentence_transformers import SentenceTransformer
import torch
import numpy as np
from tqdm import tqdm

# model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2', device="cuda:0")
# model.max_seq_length = 64
# questions = total_squad["question"]
# result = []
# with torch.no_grad():
#     for i in tqdm(range(0, len(questions), 1024), desc="Embedding..."):
#         batch = questions[i:i+1024]
#         result.extend(model.encode(batch, batch_size=1024, show_progress_bar=False))

In [49]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2', device="cuda:0")
model.max_seq_length = 64
questions = total_squad["question"]
result = []
with torch.no_grad():
    for i in tqdm(range(0, len(questions), 1024), desc="Embedding..."):
        batch = questions[i:i+1024]
        result.extend(model.encode(batch, batch_size=1024, show_progress_bar=False))

Embedding...: 100%|██████████| 29/29 [00:04<00:00,  6.99it/s]


In [50]:
samples = dataset.shuffle(seed=42).select(range(1000))

In [51]:
samples_q = samples["question"]
sampels_emb = model.encode(samples_q, batch_size=1024, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
transposed = np.array(result).T

In [54]:
from sentence_transformers import util
q_res, c_res = [], []
for q in tqdm(sampels_emb):
    max_idx = np.argmax(np.dot(q, transposed))
    max_c, max_q = total_squad["context"][max_idx], total_squad["question"][max_idx]
    q_res.append(max_q)
    c_res.append(max_c)

100%|██████████| 1000/1000 [05:57<00:00,  2.80it/s]


In [55]:
samples = samples.add_column("squad", [{"question": q_res, "context": c_res} for q_res, c_res in zip(q_res, c_res)])

In [57]:
samples.push_to_hub("Atipico1/mrqa_v2_unans_1k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [58]:
samples

Dataset({
    features: ['subset', 'qid', 'question', 'answers', 'masked_query', 'context', 'answer_sent', 'answer_in_context', 'query_embedding', 'Q_similar_context', 'C_similar_context', 'QC_similar_context', 'random_context', 'squad'],
    num_rows: 1000
})